# 加载库函数

In [1]:
import numpy as np
import scipy.sparse as sp
import networkx as nx
import networkx.algorithms.isomorphism as iso
import itertools
import os.path
import subprocess
import torch.nn.functional as F
import pickle
import os
import scipy.sparse as sp
import torch
import json
import scipy.io as sio
from torch.nn import Embedding
#import scipy.sparse as spsparse
from torch.nn import Parameter
import torch
import torch.nn as nn
from scipy.sparse import csr_matrix
import torch.nn.functional as F
import argparse
from sklearn.metrics import average_precision_score, roc_auc_score, accuracy_score, f1_score, precision_score, recall_score

# 加载数据，数据处理

In [2]:
"""
Disclaimer: functions defined from lines 15 to 36 in this file come from 
tkipf/gae original repository on Graph Autoencoders. Moreover, the
mask_test_edges_general_link_prediction function is borrowed from 
philipjackson's mask_test_edges pull request on this same repository.
"""

def sparse_to_tuple(sparse_mx):
    if not sp.isspmatrix_coo(sparse_mx):
        sparse_mx = sparse_mx.tocoo()
    coords = np.vstack((sparse_mx.row, sparse_mx.col)).transpose()
    values = sparse_mx.data
    shape = sparse_mx.shape
    return coords, values, shape

In [3]:
# Task 3 - Bidirectional Link Prediction
def mask_test_edges_bidirectional_link_prediction(adj):
    """
    Task 3: Bidirectionality Prediction: get Train and Test sets

    :param adj: complete sparse adjacency matrix of the graph
    :return: train incomplete adjacency matrix and test sets
    """

    # Remove diagonal elements
    adj = adj - sp.dia_matrix((adj.diagonal()[None, :], [0]), shape = adj.shape)
    adj.eliminate_zeros()
    val_edges, val_edges_false, test_edges, test_edges_false = None, None, None, None

    ## Unidirectional edges
    adj_sym = (adj + adj.T).sign()
    adj_tilde = (adj_sym - adj).T
    adj_tilde.eliminate_zeros()

    ## Bidirectional edges (they usually are few, so they are all in test set)
    adj_sym_ones = adj - adj_tilde
    adj_sym_ones.eliminate_zeros()

    # Positive node pairs in test set (bidirectional edges)
    test_edges, _, _ = sparse_to_tuple(adj_sym_ones)
    test_edges = test_edges[test_edges[:,1] > test_edges[:,0],:]

    # Negative node pairs in test set (unidirectional edges)
    test_edges_false, _, _ = sparse_to_tuple(adj_tilde)
    test_edges_false = test_edges_false[test_edges_false[:,0] > test_edges_false[:,1],:]
    test_edges_false = np.fliplr(test_edges_false)
    # Sampling of negative node pairs
    edges_negative_idx = np.arange(test_edges_false.shape[0])
    np.random.shuffle(edges_negative_idx)
    test_edges_false_idx = edges_negative_idx[:test_edges.shape[0]]
    test_edges_false = test_edges_false[test_edges_false_idx]

    # Get training incomplete adjacency matrix
    # 1 of the 2 directions of each bidirectional edge is masked
    adj_train = (adj - sp.triu(adj_sym_ones))
    all_train_edges = adj_train.nonzero()
    out_train_edges = [[all_train_edges[0][i],all_train_edges[1][i]] for i in range(all_train_edges[0].shape[0])]
    # Validation set: not implemented for Task 2
    # val_edges and val_edges_false are None
    return np.array(out_train_edges), val_edges_false, test_edges, test_edges_false

In [4]:
def load_data(dataset):
    """ Load datasets from text files

    :param dataset: 'cora', 'citeseer' or 'google' graph dataset.
    :return: n*n sparse adjacency matrix and n*f node-level feature matrix

    Note: in this paper, all three datasets are assumed to be featureless.
    As a consequence, the feature matrix is the identity matrix I_n.
    """
    if dataset == 'cora':
        adj = nx.adjacency_matrix(nx.read_edgelist("directed_data/cora.cites",
                                                   delimiter='\t',
                                                   create_using=nx.DiGraph()))
        # Transpose the adjacency matrix, as Cora raw dataset comes with a
        # <ID of cited paper> <ID of citing paper> edgelist format.
        adj = adj.T
        features = sp.identity(adj.shape[0])

    elif dataset == 'citeseer':
        adj = nx.adjacency_matrix(nx.read_edgelist("directed_data/citeseer.cites",
                                                   delimiter='\t',
                                                   create_using=nx.DiGraph()))
        # Transpose the adjacency matrix, as Citeseer raw dataset comes with a
        # <ID of cited paper> <ID of citing paper> edgelist format.
        adj = adj.T
        features = sp.identity(adj.shape[0])

    elif dataset in ['Wisconsin', 'Texas', 'Cornell']:
        from torch_geometric.datasets import WebKB
        from scipy import sparse

        graph_data = WebKB(root='directed_data/', name=dataset)
        data = graph_data[0]

        edges = data.edge_index
        nodes_number = data.x.shape[0]
        graph_np = np.zeros((nodes_number, nodes_number))
        for i in range(edges.shape[1]):
            graph_np[edges[0, i], edges[1, i]] = 1
    
        adj = sparse.csr_matrix(graph_np)
        
        features = sp.identity(adj.shape[0])

    else:
        raise ValueError('Undefined dataset!')

    return adj, features

In [49]:
def get_graph(DATASET='cora'):
    adj_init, features = load_data(DATASET)

    #train_edges, val_edges, val_edges_false, test_edges, test_edges_false = mask_test_edges_biased_negative_samples(adj_init)
    train_edges, val_edges_false, test_edges, test_edges_false = mask_test_edges_bidirectional_link_prediction(adj_init)
    
    all_negative_edges = np.argwhere(adj_init.toarray()==0)
    
    all_train_edges_false = np.delete(all_negative_edges, test_edges_false, axis = 0)
    
    if DATASET == 'cora':
        nodes_number = 2708
    elif DATASET == 'citeseer':
        nodes_number = 3327
    elif DATASET == 'Wisconsin':
        nodes_number = 251
    elif DATASET == 'Texas':
        nodes_number = 183
    elif DATASET == 'Cornell':
        nodes_number = 183
        
    ################## 处理测试集  #############################
    test_posi_mask = [test_edges[i][0]*nodes_number+test_edges[i][1] for i in range(len(test_edges))]
    test_false_mask = [test_edges_false[i][0]*nodes_number+test_edges_false[i][1] for i in range(len(test_edges_false))]
    test_mask = test_posi_mask + test_false_mask
    test_mask = torch.tensor(test_mask)
    test_edge_true_or_false = torch.zeros(len(test_mask))
    test_edge_true_or_false[:len(test_posi_mask)] = 1
    
    ############### 处理训练集  ################################
    negative_index = np.random.choice(range(all_train_edges_false.shape[0]), train_edges.shape[0]*2, replace=False)
    train_edges_false = all_train_edges_false[negative_index]
    train_posi_mask = [train_edges[i][0]*nodes_number+train_edges[i][1] for i in range(len(train_edges))]
    train_false_mask = [train_edges_false[i][0]*nodes_number+train_edges_false[i][1] for i in range(len(train_edges_false))]
    train_mask = train_posi_mask + train_false_mask
    train_mask = torch.tensor(train_mask)
    #train_mask = torch.tensor(train_posi_mask)
    train_edge_true_or_false = torch.zeros(len(train_mask))
    train_edge_true_or_false[:len(train_posi_mask)] = 1
    
    return train_edge_true_or_false, test_edge_true_or_false, train_mask.long(), test_mask.long(), nodes_number

In [50]:
#train_edge, test_edge, train_mask, test_mask, nodes_number = get_graph(task=2)

# 定义模型

In [51]:
def predict_acc(recons_edges, true_edges):
    predict_graph = recons_edges
    predict_edges = np.array(predict_graph)
    ap = average_precision_score(true_edges, predict_edges)
    auc = roc_auc_score(true_edges, predict_edges)
    print("AP SCORE： ", ap)
    print("AUC SCORE: ", auc)
    return ap, auc

################  更新节点内容  #######################################
class reconstruction_graph(nn.Module):
    """给定类簇质心，更新节点嵌入"""
    def __init__(self, context_embedding):
        super(reconstruction_graph, self).__init__()
        self.context_embedding = Parameter(context_embedding)
        
    def forward(self, importance_embedding):
        power = float(ALPHA + 1) / 2    # 计算幂
        
        # 计算节点之间的距离 ： 值越小越有利于边的形成
        nodes_distance = torch.norm(self.context_embedding[:, None]-self.context_embedding, dim=2, p=2)
        nodes_distance = 1.0 + nodes_distance
        
        # 计算节点之间的重要性：值越大越有利于边的形成
        norm_squared = torch.sum((self.context_embedding.unsqueeze(1) - importance_embedding)**2, 2)
        nodes_importance = N / (1.0 + norm_squared) + 1.0
        importance_prod = torch.mm(nodes_importance, nodes_importance.t())   # 分子
        importance_norm = torch.norm(nodes_importance, p=2, dim=1).unsqueeze(0)   #分母(每行的长度)
        importance_norm = importance_norm**2
        importance_similar = importance_prod.div(importance_norm.t())
        
        # 计算边的形成概率
        nodes_similar = torch.div(beta*importance_similar, nodes_distance)
        return 2*sigmoid_modle(nodes_similar)-1
        
    
class update_nodes_embedding(nn.Module):
    def __init__(self, context_embedding):
        super(update_nodes_embedding, self).__init__()
        self.reconstruction_module = reconstruction_graph(context_embedding)     # 更新节点嵌入
        self.optimizer = torch.optim.SGD(params=self.reconstruction_module.parameters(), lr=LR, momentum=MOMENTUM)
        self.loss_function = torch.nn.MSELoss(reduction='sum')
        
    def forward(self, train_edges, test_edges, edge_train_mask, edge_test_mask, importance_embedding):
        self.reconstruction_module.train()
        for epoch in range(EPOCH):
            self.optimizer.zero_grad()
            graph_reconstruction = self.reconstruction_module(importance_embedding)
            graph_train = torch.take(graph_reconstruction, edge_train_mask)
            loss = self.loss_function(train_edges, graph_train)
            loss.backward()
            self.optimizer.step()
            print(f'Epoch: {epoch:02d}, Loss: {loss.item():.4f}')
        recons_test_edges = torch.take(graph_reconstruction, edge_test_mask).detach()
        ap, auc = predict_acc(recons_test_edges, test_edges)
        return self.reconstruction_module.context_embedding.detach(), ap, auc

####################  更新节点重要性  ######################################
class reconstruction_graph_importance(nn.Module):
    """给定类簇质心，更新节点嵌入"""
    def __init__(self, importance_embedding):
        super(reconstruction_graph_importance, self).__init__()
        self.importance_embedding = Parameter(importance_embedding)
        
    def forward(self, context_embedding):
        power = float(ALPHA + 1) / 2    # 计算幂
        
        # 计算节点之间的距离 ： 值越小越有利于边的形成
        nodes_distance = torch.norm(context_embedding[:, None]-context_embedding, dim=2, p=2)
        nodes_distance = 1.0 + nodes_distance
        
        # 计算节点之间的重要性：值越大越有利于边的形成
        norm_squared = torch.sum((context_embedding.unsqueeze(1) - self.importance_embedding)**2, 2)
        nodes_importance = N / (1.0 + norm_squared) + 1.0
        importance_prod = torch.mm(nodes_importance, nodes_importance.t())   # 分子
        importance_norm = torch.norm(nodes_importance, p=2, dim=1).unsqueeze(0)   #分母(每行的长度)
        importance_norm = importance_norm**2
        importance_similar = importance_prod.div(importance_norm.t())
        
        # 计算边的形成概率
        nodes_similar = torch.div(beta*importance_similar, nodes_distance)
        return 2*sigmoid_modle(nodes_similar)-1

class update_nodes_embedding_importance(nn.Module):
    def __init__(self, importance_embedding):
        super(update_nodes_embedding_importance, self).__init__()
        self.reconstruction_module = reconstruction_graph_importance(importance_embedding)     # 更新节点嵌入
        self.optimizer = torch.optim.SGD(params=self.reconstruction_module.parameters(), lr=LR, momentum=MOMENTUM)
        self.loss_function = torch.nn.MSELoss(reduction='sum')
        
    def forward(self, train_edges, test_edges, edge_train_mask, edge_test_mask, context_embedding):
        self.reconstruction_module.train()
        for epoch in range(EPOCH):
            self.optimizer.zero_grad()
            graph_reconstruction = self.reconstruction_module(context_embedding)
            graph_train = torch.take(graph_reconstruction, edge_train_mask)
            loss = self.loss_function(train_edges, graph_train)
            loss.backward()
            self.optimizer.step()
            print(f'Epoch: {epoch:02d}, Loss: {loss.item():.4f}')
        recons_test_edges = torch.take(graph_reconstruction, edge_test_mask).detach()
        ap, auc = predict_acc(recons_test_edges, test_edges)
        return self.reconstruction_module.importance_embedding.detach(), ap, auc

In [61]:
SEED = 1234
np.random.seed(SEED)
torch.manual_seed(SEED)

context_embedding_dim = 12
importance_embedding_dim = 48
beta = 5.0
ALPHA = 1.0
LR = 0.01
MOMENTUM = 0.9
EPOCH = 10
N = 29.0    # 目前25最佳
sigmoid_modle = torch.nn.Sigmoid()

dataset = 'Cornell'

In [62]:
all_best_auc = []

all_best_ap = []

for i in range(10):
    
    train_edge, test_edge, train_mask, test_mask, nodes_number = get_graph(DATASET=dataset)
    
    context_embedding = Embedding(nodes_number, context_embedding_dim, sparse=True)      # 时刻为0时给定随机的嵌入
    
    nodes_context_embeddings = context_embedding.weight.detach()

    importance_embedding = Embedding(importance_embedding_dim, context_embedding_dim, sparse=True)      # 时刻为0时给定随机的嵌入
    
    nodes_importance_embeddings = importance_embedding.weight.detach()
    
    auc_first_best_auc = 0
    auc_first_best_ap = 0
    ap_first_best_auc = 0
    ap_first_best_ap = 0

    for module_epoch in range(200):
        print("######################### 模块循环 ： %d ##########################"%module_epoch)
        update_nodes_module = update_nodes_embedding(nodes_context_embeddings)
        nodes_context_embeddings, ap, auc = update_nodes_module(train_edge, test_edge, train_mask, test_mask, nodes_importance_embeddings)
        if auc > auc_first_best_auc:
            auc_first_best_auc = auc
            auc_first_best_ap = ap
        if ap > ap_first_best_ap:
            ap_first_best_auc = auc
            ap_first_best_ap = ap
        update_nodes_importance_module = update_nodes_embedding_importance(nodes_importance_embeddings)
        nodes_importance_embeddings, ap, auc = update_nodes_importance_module(train_edge, test_edge, train_mask, test_mask, nodes_context_embeddings)
        if auc > auc_first_best_auc:
            auc_first_best_auc = auc
            auc_first_best_ap = ap
        if ap > ap_first_best_ap:
            ap_first_best_auc = auc
            ap_first_best_ap = ap
    all_best_ap.append(auc_first_best_ap)
    all_best_auc.append(auc_first_best_auc)

######################### 模块循环 ： 0 ##########################
Epoch: 00, Loss: 197.5986
Epoch: 01, Loss: 197.4248
Epoch: 02, Loss: 197.0965
Epoch: 03, Loss: 196.6320
Epoch: 04, Loss: 196.0494
Epoch: 05, Loss: 195.3652
Epoch: 06, Loss: 194.5950
Epoch: 07, Loss: 193.7530
Epoch: 08, Loss: 192.8518
Epoch: 09, Loss: 191.9030
AP SCORE：  0.5216270241492469
AUC SCORE:  0.521604938271605
Epoch: 00, Loss: 190.9166
Epoch: 01, Loss: 190.8952
Epoch: 02, Loss: 190.8545
Epoch: 03, Loss: 190.7967
Epoch: 04, Loss: 190.7238
Epoch: 05, Loss: 190.6375
Epoch: 06, Loss: 190.5397
Epoch: 07, Loss: 190.4317
Epoch: 08, Loss: 190.3150
Epoch: 09, Loss: 190.1907
AP SCORE：  0.5271039160844722
AUC SCORE:  0.5216049382716049
######################### 模块循环 ： 1 ##########################
Epoch: 00, Loss: 190.0601
Epoch: 01, Loss: 189.9192
Epoch: 02, Loss: 189.6524
Epoch: 03, Loss: 189.2740
Epoch: 04, Loss: 188.7971
Epoch: 05, Loss: 188.2342
Epoch: 06, Loss: 187.5969
Epoch: 07, Loss: 186.8958
Epoch: 08, Loss: 186.1406
E

Epoch: 07, Loss: 143.2883
Epoch: 08, Loss: 143.1997
Epoch: 09, Loss: 143.1059
AP SCORE：  0.6894488304096
AUC SCORE:  0.6481481481481481
######################### 模块循环 ： 12 ##########################
Epoch: 00, Loss: 143.0080
Epoch: 01, Loss: 142.9617
Epoch: 02, Loss: 142.8740
Epoch: 03, Loss: 142.7495
Epoch: 04, Loss: 142.5922
Epoch: 05, Loss: 142.4062
Epoch: 06, Loss: 142.1950
Epoch: 07, Loss: 141.9619
Epoch: 08, Loss: 141.7100
Epoch: 09, Loss: 141.4418
AP SCORE：  0.6857204982290891
AUC SCORE:  0.6450617283950617
Epoch: 00, Loss: 141.1600
Epoch: 01, Loss: 141.1451
Epoch: 02, Loss: 141.1166
Epoch: 03, Loss: 141.0763
Epoch: 04, Loss: 141.0255
Epoch: 05, Loss: 140.9655
Epoch: 06, Loss: 140.8976
Epoch: 07, Loss: 140.8229
Epoch: 08, Loss: 140.7424
Epoch: 09, Loss: 140.6570
AP SCORE：  0.6868151889789522
AUC SCORE:  0.6481481481481481
######################### 模块循环 ： 13 ##########################
Epoch: 00, Loss: 140.5676
Epoch: 01, Loss: 140.5249
Epoch: 02, Loss: 140.4440
Epoch: 03, Loss: 1

Epoch: 05, Loss: 121.0681
Epoch: 06, Loss: 121.0353
Epoch: 07, Loss: 120.9993
Epoch: 08, Loss: 120.9605
Epoch: 09, Loss: 120.9194
AP SCORE：  0.7382236237194115
AUC SCORE:  0.7098765432098766
######################### 模块循环 ： 25 ##########################
Epoch: 00, Loss: 120.8764
Epoch: 01, Loss: 120.8531
Epoch: 02, Loss: 120.8090
Epoch: 03, Loss: 120.7462
Epoch: 04, Loss: 120.6670
Epoch: 05, Loss: 120.5731
Epoch: 06, Loss: 120.4663
Epoch: 07, Loss: 120.3482
Epoch: 08, Loss: 120.2202
Epoch: 09, Loss: 120.0836
AP SCORE：  0.7426466441146542
AUC SCORE:  0.712962962962963
Epoch: 00, Loss: 119.9396
Epoch: 01, Loss: 119.9325
Epoch: 02, Loss: 119.9192
Epoch: 03, Loss: 119.9003
Epoch: 04, Loss: 119.8765
Epoch: 05, Loss: 119.8485
Epoch: 06, Loss: 119.8168
Epoch: 07, Loss: 119.7821
Epoch: 08, Loss: 119.7448
Epoch: 09, Loss: 119.7053
AP SCORE：  0.7426466441146542
AUC SCORE:  0.712962962962963
######################### 模块循环 ： 26 ##########################
Epoch: 00, Loss: 119.6640
Epoch: 01, Loss: 

Epoch: 00, Loss: 108.0189
Epoch: 01, Loss: 108.0144
Epoch: 02, Loss: 108.0059
Epoch: 03, Loss: 107.9939
Epoch: 04, Loss: 107.9788
Epoch: 05, Loss: 107.9612
Epoch: 06, Loss: 107.9413
Epoch: 07, Loss: 107.9197
Epoch: 08, Loss: 107.8965
Epoch: 09, Loss: 107.8721
AP SCORE：  0.7654786598033725
AUC SCORE:  0.7345679012345678
######################### 模块循环 ： 38 ##########################
Epoch: 00, Loss: 107.8468
Epoch: 01, Loss: 107.8299
Epoch: 02, Loss: 107.7977
Epoch: 03, Loss: 107.7519
Epoch: 04, Loss: 107.6940
Epoch: 05, Loss: 107.6253
Epoch: 06, Loss: 107.5471
Epoch: 07, Loss: 107.4606
Epoch: 08, Loss: 107.3668
Epoch: 09, Loss: 107.2665
AP SCORE：  0.769039913364626
AUC SCORE:  0.7376543209876543
Epoch: 00, Loss: 107.1606
Epoch: 01, Loss: 107.1560
Epoch: 02, Loss: 107.1474
Epoch: 03, Loss: 107.1352
Epoch: 04, Loss: 107.1200
Epoch: 05, Loss: 107.1021
Epoch: 06, Loss: 107.0819
Epoch: 07, Loss: 107.0600
Epoch: 08, Loss: 107.0365
Epoch: 09, Loss: 107.0119
AP SCORE：  0.7677578620825747
AUC SC

Epoch: 09, Loss: 97.3468
AP SCORE：  0.7733675559838891
AUC SCORE:  0.7345679012345679
Epoch: 00, Loss: 97.2525
Epoch: 01, Loss: 97.2474
Epoch: 02, Loss: 97.2378
Epoch: 03, Loss: 97.2243
Epoch: 04, Loss: 97.2073
Epoch: 05, Loss: 97.1873
Epoch: 06, Loss: 97.1649
Epoch: 07, Loss: 97.1403
Epoch: 08, Loss: 97.1140
Epoch: 09, Loss: 97.0862
AP SCORE：  0.7733675559838891
AUC SCORE:  0.7345679012345679
######################### 模块循环 ： 51 ##########################
Epoch: 00, Loss: 97.0572
Epoch: 01, Loss: 97.0419
Epoch: 02, Loss: 97.0128
Epoch: 03, Loss: 96.9715
Epoch: 04, Loss: 96.9193
Epoch: 05, Loss: 96.8576
Epoch: 06, Loss: 96.7874
Epoch: 07, Loss: 96.7099
Epoch: 08, Loss: 96.6260
Epoch: 09, Loss: 96.5365
AP SCORE：  0.7733675559838891
AUC SCORE:  0.7345679012345679
Epoch: 00, Loss: 96.4423
Epoch: 01, Loss: 96.4368
Epoch: 02, Loss: 96.4265
Epoch: 03, Loss: 96.4119
Epoch: 04, Loss: 96.3936
Epoch: 05, Loss: 96.3721
Epoch: 06, Loss: 96.3478
Epoch: 07, Loss: 96.3212
Epoch: 08, Loss: 96.2926
Epoc

...

######################### 模块循环 ： 148 ##########################
Epoch: 00, Loss: 53.8090
Epoch: 01, Loss: 53.8056
Epoch: 02, Loss: 53.7992
Epoch: 03, Loss: 53.7901
Epoch: 04, Loss: 53.7786
Epoch: 05, Loss: 53.7651
Epoch: 06, Loss: 53.7497
Epoch: 07, Loss: 53.7327
Epoch: 08, Loss: 53.7144
Epoch: 09, Loss: 53.6950
AP SCORE：  0.718057670361592
AUC SCORE:  0.6759259259259258
Epoch: 00, Loss: 53.6745
Epoch: 01, Loss: 53.6739
Epoch: 02, Loss: 53.6728
Epoch: 03, Loss: 53.6713
Epoch: 04, Loss: 53.6694
Epoch: 05, Loss: 53.6673
Epoch: 06, Loss: 53.6649
Epoch: 07, Loss: 53.6625
Epoch: 08, Loss: 53.6599
Epoch: 09, Loss: 53.6573
AP SCORE：  0.704168781472703
AUC SCORE:  0.6728395061728394
######################### 模块循环 ： 149 ##########################
Epoch: 00, Loss: 53.6547
Epoch: 01, Loss: 53.6514
Epoch: 02, Loss: 53.6452
Epoch: 03, Loss: 53.6363
Epoch: 04, Loss: 53.6251
Epoch: 05, Loss: 53.6119
Epoch: 06, Loss: 53.5969
Epoch: 07, Loss: 53.5803
Epoch: 08, Loss: 53.5625
Epoch: 09, Loss: 53.5435
AP

Epoch: 01, Loss: 52.0103
Epoch: 02, Loss: 52.0092
Epoch: 03, Loss: 52.0076
Epoch: 04, Loss: 52.0057
Epoch: 05, Loss: 52.0035
Epoch: 06, Loss: 52.0011
Epoch: 07, Loss: 51.9985
Epoch: 08, Loss: 51.9958
Epoch: 09, Loss: 51.9931
AP SCORE：  0.7098399697010808
AUC SCORE:  0.6851851851851852
######################### 模块循环 ： 161 ##########################
Epoch: 00, Loss: 51.9903
Epoch: 01, Loss: 51.9876
Epoch: 02, Loss: 51.9824
Epoch: 03, Loss: 51.9750
Epoch: 04, Loss: 51.9657
Epoch: 05, Loss: 51.9547
Epoch: 06, Loss: 51.9422
Epoch: 07, Loss: 51.9285
Epoch: 08, Loss: 51.9136
Epoch: 09, Loss: 51.8978
AP SCORE：  0.7098399697010808
AUC SCORE:  0.6851851851851852
Epoch: 00, Loss: 51.8812
Epoch: 01, Loss: 51.8806
Epoch: 02, Loss: 51.8794
Epoch: 03, Loss: 51.8779
Epoch: 04, Loss: 51.8759
Epoch: 05, Loss: 51.8737
Epoch: 06, Loss: 51.8712
Epoch: 07, Loss: 51.8686
Epoch: 08, Loss: 51.8659
Epoch: 09, Loss: 51.8631
AP SCORE：  0.7098399697010808
AUC SCORE:  0.6851851851851852
######################### 模块

Epoch: 02, Loss: 50.3958
Epoch: 03, Loss: 50.3884
Epoch: 04, Loss: 50.3791
Epoch: 05, Loss: 50.3680
Epoch: 06, Loss: 50.3555
Epoch: 07, Loss: 50.3416
Epoch: 08, Loss: 50.3267
Epoch: 09, Loss: 50.3107
AP SCORE：  0.6534042355479553
AUC SCORE:  0.6759259259259258
Epoch: 00, Loss: 50.2940
Epoch: 01, Loss: 50.2930
Epoch: 02, Loss: 50.2913
Epoch: 03, Loss: 50.2889
Epoch: 04, Loss: 50.2859
Epoch: 05, Loss: 50.2824
Epoch: 06, Loss: 50.2785
Epoch: 07, Loss: 50.2742
Epoch: 08, Loss: 50.2697
Epoch: 09, Loss: 50.2649
AP SCORE：  0.6534042355479553
AUC SCORE:  0.6759259259259258
######################### 模块循环 ： 174 ##########################
Epoch: 00, Loss: 50.2600
Epoch: 01, Loss: 50.2573
Epoch: 02, Loss: 50.2520
Epoch: 03, Loss: 50.2446
Epoch: 04, Loss: 50.2352
Epoch: 05, Loss: 50.2241
Epoch: 06, Loss: 50.2115
Epoch: 07, Loss: 50.1976
Epoch: 08, Loss: 50.1825
Epoch: 09, Loss: 50.1664
AP SCORE：  0.6534042355479553
AUC SCORE:  0.6759259259259258
Epoch: 00, Loss: 50.1495
Epoch: 01, Loss: 50.1486
Epo

Epoch: 03, Loss: 48.6035
Epoch: 04, Loss: 48.6009
Epoch: 05, Loss: 48.5979
Epoch: 06, Loss: 48.5945
Epoch: 07, Loss: 48.5909
Epoch: 08, Loss: 48.5871
Epoch: 09, Loss: 48.5832
AP SCORE：  0.6447420121031232
AUC SCORE:  0.675925925925926
######################### 模块循环 ： 186 ##########################
Epoch: 00, Loss: 48.5792
Epoch: 01, Loss: 48.5766
Epoch: 02, Loss: 48.5716
Epoch: 03, Loss: 48.5646
Epoch: 04, Loss: 48.5557
Epoch: 05, Loss: 48.5452
Epoch: 06, Loss: 48.5333
Epoch: 07, Loss: 48.5201
Epoch: 08, Loss: 48.5059
Epoch: 09, Loss: 48.4908
AP SCORE：  0.6447420121031232
AUC SCORE:  0.675925925925926
Epoch: 00, Loss: 48.4750
Epoch: 01, Loss: 48.4742
Epoch: 02, Loss: 48.4727
Epoch: 03, Loss: 48.4706
Epoch: 04, Loss: 48.4681
Epoch: 05, Loss: 48.4651
Epoch: 06, Loss: 48.4618
Epoch: 07, Loss: 48.4583
Epoch: 08, Loss: 48.4545
Epoch: 09, Loss: 48.4507
AP SCORE：  0.6447420121031232
AUC SCORE:  0.675925925925926
######################### 模块循环 ： 187 ##########################
Epoch: 00, Loss: 

Epoch: 01, Loss: 47.0533
Epoch: 02, Loss: 47.0485
Epoch: 03, Loss: 47.0418
Epoch: 04, Loss: 47.0332
Epoch: 05, Loss: 47.0232
Epoch: 06, Loss: 47.0117
Epoch: 07, Loss: 46.9990
Epoch: 08, Loss: 46.9854
Epoch: 09, Loss: 46.9708
AP SCORE：  0.6287697898809009
AUC SCORE:  0.6728395061728395
Epoch: 00, Loss: 46.9554
Epoch: 01, Loss: 46.9549
Epoch: 02, Loss: 46.9539
Epoch: 03, Loss: 46.9524
Epoch: 04, Loss: 46.9506
Epoch: 05, Loss: 46.9486
Epoch: 06, Loss: 46.9463
Epoch: 07, Loss: 46.9439
Epoch: 08, Loss: 46.9414
Epoch: 09, Loss: 46.9389
AP SCORE：  0.6287697898809009
AUC SCORE:  0.6728395061728395
######################### 模块循环 ： 199 ##########################
Epoch: 00, Loss: 46.9363
Epoch: 01, Loss: 46.9338
Epoch: 02, Loss: 46.9291
Epoch: 03, Loss: 46.9224
Epoch: 04, Loss: 46.9140
Epoch: 05, Loss: 46.9040
Epoch: 06, Loss: 46.8926
Epoch: 07, Loss: 46.8801
Epoch: 08, Loss: 46.8666
Epoch: 09, Loss: 46.8521
AP SCORE：  0.6287697898809009
AUC SCORE:  0.6728395061728395
Epoch: 00, Loss: 46.8370
Epo

In [63]:
print("MEAN AP : ", np.array(all_best_ap).mean())
print("STD AP : ", np.array(all_best_ap).std())
print("MEAN AUC : ", np.array(all_best_auc).mean())
print("STD AUC : ", np.array(all_best_auc).std())

"""
print("AUC FIRST BEST AUC: ", auc_first_best_auc)
print("AUC FIRST BEST AP: ", auc_first_best_ap)
print("AP FIRST BEST AUC: ", ap_first_best_auc)
print("AP FIRST BEST AP: ", ap_first_best_ap)
"""

MEAN AP :  0.7821952640989862
STD AP :  0.0767252589949115
MEAN AUC :  0.7731481481481481
STD AUC :  0.06262086317028404


'\nprint("AUC FIRST BEST AUC: ", auc_first_best_auc)\nprint("AUC FIRST BEST AP: ", auc_first_best_ap)\nprint("AP FIRST BEST AUC: ", ap_first_best_auc)\nprint("AP FIRST BEST AP: ", ap_first_best_ap)\n'